In [27]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np

In [4]:
root_folder = '../CDSL_HM_20200720_RAW'

TODO: informacion de los datos

In [49]:
def set_display_range(range_columns=pd.options.display.max_columns, range_rows=pd.options.display.max_rows, reset_columns=0, reset_rows=0):
    pd.set_option('display.max_columns', range_columns)
    pd.set_option('display.max_rows', range_rows)
    if (reset_columns): pd.reset_option('display.max_columns')
    if (reset_rows): pd.reset_option('display.max_rows')

In [69]:
def column_percentage(df, column, main_label):
    try:
        return len(df.loc[df[column]==main_label])/len(df)
    except:
        return -1

In [8]:
# CDSL_01
df_patient = pd.read_csv(root_folder+'/CDSL_01.csv', sep=';')
# CDSL_02
df_med = pd.read_csv(root_folder+'/CDSL_02.csv', sep=';')#, encoding='iso-8859-1')
# CDSL_03
df_vital = pd.read_csv(root_folder+'/CDSL_03.csv', sep=';')
# CDSL_05 (emergency)
df_ICD10_emer = pd.read_csv(root_folder+'/CDSL_05.csv', sep=';')
# CDSL_06  (admission)
df_ICD10_adm = pd.read_csv(root_folder+'/CDSL_06.csv', sep=';')

In [9]:
# CDSL_04
df_lab = pd.read_csv(root_folder+'/CDSL_04.csv', sep=';', error_bad_lines=False, encoding='iso-8859-1')

b'Skipping line 28039: expected 8 fields, saw 12\nSkipping line 32182: expected 8 fields, saw 12\nSkipping line 44299: expected 8 fields, saw 12\n'
b'Skipping line 90790: expected 8 fields, saw 12\nSkipping line 105234: expected 8 fields, saw 12\nSkipping line 130735: expected 8 fields, saw 10\n'
b'Skipping line 136659: expected 8 fields, saw 10\nSkipping line 140398: expected 8 fields, saw 10\nSkipping line 141687: expected 8 fields, saw 12\nSkipping line 142745: expected 8 fields, saw 12\nSkipping line 170183: expected 8 fields, saw 12\n'
b'Skipping line 223336: expected 8 fields, saw 21\nSkipping line 237392: expected 8 fields, saw 12\n'
b'Skipping line 373889: expected 8 fields, saw 12\nSkipping line 382978: expected 8 fields, saw 10\n'
b'Skipping line 412136: expected 8 fields, saw 12\nSkipping line 419764: expected 8 fields, saw 12\nSkipping line 421226: expected 8 fields, saw 12\nSkipping line 426429: expected 8 fields, saw 11\nSkipping line 444865: expected 8 fields, saw 12\n'


In [59]:
df_patient =df_patient.rename(columns=\
                        {"PATIENT ID":"PATIENT_ID", "EDAD/AGE":"AGE", "SEXO/SEX":"SEX", "DIAG ING/INPAT":"DIAG_INPAT", \
                        "F_INGRESO/ADMISSION_D_ING/INPAT":"ADMISSION_DATE_INPAT","HORA/TIME_ADMISION/ADMISSION_URG/EMERG":\
                         "TIME_ADMISSION_EMERG","F_ENTRADA_UC/ICU_DATE_IN":"ICU_DATE_IN", \
                        "F_SALIDA_UCI/ICU_DATE_OUT":"ICU_DATE_OUT","UCI_DIAS/ICU_DAYS": "ICU_DAYS", \
                        "RESPIRADOR/MECH.VENT.":"MECH.VENT.", "F_ALTA/DISCHARGE_DATE_ING":"DISCHARGE_DATE_ING", \
                        "MOTIVO_ALTA/DESTINY_DISCHARGE_ING":"DESTINY_DISCHARGE_ING","F_INGRESO/ADMISSION_DATE_URG/EMERG":
                        "ADMISSION_DATE_EMERG", "HORA/TIME_ADMISION/ADMISSION_EMERG/EMERG":"TIME_ADMISION_EMERG", \
                        "ESPECIALIDAD/DEPARTMENT_URG/EMERG":"DEPARTMENT_EMERG","DIAG_URG/EMERG":"DIAG_EMERG", \
                        "DESTINO/DESTINY_URG/EMERG":"DESTINY_EMERG", "HORA/TIME_CONSTANT_PRIMERA/FIRST_URG/EMERG" : \
                        "TIME_CONSTANT_FIRST_EMERG", "TEMP_PRIMERA/FIRST_URG/EMERG":"TEMP_FIRST_EMERG", "FC/HR_PRIMERA/FIRST_URG/EMERG": \
                        "HR_FIRST_EMERG", "GLU_PRIMERA/FIRST_URG/EMERG":"GLU_FIRST_EMERG", "SAT_02_PRIMERA/FIRST_URG/EMERG":\
                        "SAT_02_FIRST_EMERG", "TA_MAX_PRIMERA/FIRST/EMERG_URG":"TA_MAX_FIRST_EMERG", "TA_MIN_PRIMERA/FIRST_URG/EMERG":\
                        "TA_MIN_FIRST_EMERG", "HORA/TIME_CONSTANT_ULTIMA/LAST_URG/EMERG":"TIME_CONSTANT_LAST_EMERG", \
                        "FC/HR_ULTIMA/LAST_URG/EMERG":"HR_LAST_EMERG", "TEMP_ULTIMA/LAST_URG/EMERG":"TEMP_LAST_EMERG", \
                        "GLU_ULTIMA/LAST_URG/EMERG":"GLU_LAST_EMERG", "SAT_02_ULTIMA/LAST_URG/EMERG":"SAT_02_LAST_EMERG", \
                        "TA_MAX_ULTIMA/LAST_URGEMERG":"TA_MAX_LAST_MERG", "TA_MIN_ULTIMA/LAST_URG/EMERG":"TA_MIN_LAST_EMERG"})
                        


In [67]:
len(df_patient)

2547

In [60]:
set_display_range(reset_rows=1)
df_patient

,PATIENT_ID,AGE,SEX,DIAG_INPAT,ADMISSION_DATE_INPAT,ICU_DATE_IN,ICU_DATE_OUT,ICU_DAYS,MECH.VENT.,DISCHARGE_DATE_ING,DESTINY_DISCHARGE_ING,ADMISSION_DATE_EMERG,TIME_ADMISSION_EMERG,DEPARTMENT_EMERG,DIAG_EMERG,DESTINY_EMERG,TIME_CONSTANT_FIRST_EMERG,TEMP_FIRST_EMERG,HR_FIRST_EMERG,GLU_FIRST_EMERG,SAT_02_FIRST_EMERG,TA_MAX_FIRST_EMERG,TA_MIN_FIRST_EMERG,TIME_CONSTANT_LAST_EMERG,HR_LAST_EMERG,TEMP_LAST_EMERG,GLU_LAST_EMERG,SAT_02_LAST_EMERG,TA_MAX_LAST_MERG,TA_MIN_LAST_EMERG
0,1,85,MALE,COVID CONFIRMADO,2020-04-06,NaN,NaN,NaN,SI,2020-04-11,Fallecimiento,2020-04-06,1899-12-30 19:05:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,1899-12-30 19:16:00,36.1,67,0,95,147,68,1899-12-30 19:12:00,67,36.099998,0,95,147,68
1,2,0,MALE,COVID SOSPECHA,2020-04-08,2020-04-08 14:54:38,2020-04-10 21:24:00,2.0,NaN,2020-04-10,Domicilio,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,NaN,0,0.000000,0,0,0,0
2,3,0,FEMALE,COVID SOSPECHA,2020-04-09,2020-04-09 23:23:09,2020-04-10 21:22:03,1.0,NaN,2020-04-10,Domicilio,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,NaN,0,0.000000,0,0,0,0
3,4,0,MALE,COVID SOSPECHA,2020-04-10,2020-04-10 01:56:08,2020-04-10 21:23:03,0.0,NaN,2020-04-10,Domicilio,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,NaN,0,0.000000,0,0,0,0
4,5,0,FEMALE,COVID SOSPECHA,2020-04-10,2020-04-10 02:14:24,2020-04-11 20:14:02,1.0,NaN,2020-04-11,Domicilio,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,NaN,0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,2567,53,MALE,COVID CONFIRMADO,2020-04-22,NaN,NaN,NaN,SI,2020-04-25,Domicilio,2020-04-22,1899-12-30 18:23:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,NaN,0.0,0,0,0,0,0,1899-12-30 18:28:00,0,0.000000,0,0,0,0
2543,2568,88,FEMALE,COVID CONFIRMADO,2020-04-22,NaN,NaN,NaN,NaN,2020-04-23,Domicilio,2020-04-22,1899-12-30 19:48:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,1899-12-30 19:56:00,37.1,86,0,98,136,65,1899-12-30 20:16:00,0,0.000000,0,0,0,0
2544,2569,81,FEMALE,COVID CONFIRMADO,2020-04-22,NaN,NaN,NaN,SI,2020-04-30,Domicilio,2020-04-22,1899-12-30 18:30:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,1899-12-30 18:37:00,36.4,59,0,96,159,70,1899-12-30 19:32:00,59,36.400002,0,96,159,70
2545,2570,31,FEMALE,COVID CONFIRMADO,2020-04-23,NaN,NaN,NaN,NaN,2020-04-24,Domicilio,2020-04-22,1899-12-30 22:39:00,Medicina de Urgencias,FIEBRE,Ingreso,1899-12-30 22:49:00,37.0,108,0,98,110,90,1899-12-30 01:33:00,108,0.000000,0,98,110,90


In [76]:
print("MALE/FEMALE %: {:.2f}%".format(column_percentage(df_patient,'SEX','MALE')))
print("CONFIRMED COVID/SUSPECTED COVID %: {:.2f}%".format(column_percentage(df_patient,'DIAG_INPAT','COVID CONFIRMADO')))
print("DECEASED/REST %: {:.2f}%".format(column_percentage(df_patient,'DESTINY_DISCHARGE_ING','Fallecimiento')))

MALE/FEMALE %: 0.59%
CONFIRMED COVID/SUSPECTED COVID %: 0.91%
DECEASED/REST %: 0.15%


In [66]:
# len(df_patient.loc[df_patient['ICU_DAYS'].isnull()])
df_patient['DIAG_INPAT'].unique()
df_patient['DESTINY_DISCHARGE_ING'].unique()
df_patient['DIAG_EMERG'].unique()

array(['DIFICULTAD RESPIRATORIA', nan, 'CUADRO CATARRAL', 'FIEBRE', 'TOS',
       'DISURIA', 'SÍNCOPE', 'MALESTAR GENERAL', 'DESORIENTACIÓN',
       'TRAUMATISMO CRANEAL', 'DIARREA', 'DETERIORO PACIENTE ONCOLÓGICO',
       'ASTENIA', 'TRAUMATISMO MMII', 'NÁUSEAS/VÓMITOS', 'MALESTAR',
       'DOLOR ABDOMINAL', 'MAREO', 'DOLOR TORÁCICO', 'TAQUICARDIA',
       'ALTERACIÓN DEL NIVEL DE CONCIENCIA', 'HIPERTENSIÓN ARTERIAL',
       'PARESIA MMSS', 'DOLOR COSTAL', 'DOLOR AL TRAGAR',
       'DIFICULTAD EN DIURESIS', 'DOLOR MMII', 'TRAUMATISMO FACIAL',
       'RECTORRAGIA', 'HIPERGLUCEMIA', 'REACCIÓN ALÉRGICA',
       'TUMORACIÓN (SIN ESPECIFICAR)', 'MELENAS', 'TRAUMATISMO COSTAL',
       'SOSPECHA CÓLICO NEFRITICO', 'DOLOR LUMBAR',
       'PÉRDIDA DE CONOCIMIENTO', 'HEMORRAGIA DIGESTIVA',
       'HINCHAZÓN MMII', 'DIFICULTAD PARA HABLAR',
       'PALPITACIONES (ARRITMIA)', 'HERIDA CABEZA/CARA', 'SOSPECHA RPM',
       'COMPLICACIÓN HERIDA QUIRÚRGICA', 'DOLOR EN EL PECHO', 'EPISTAXIS',
       'M